In [1]:
# This script will do all necessary preprocessing for daily data to be scored by AD models

# 1. Reading all of one days data from datalake for choosen station
# 2. Implement deadband-filter on data
# 3. Create and output preprocessed dataset and related features
# 4. Create and output plateaus and related features
# 5. Create and output passages and related features


# Change log
# Date Initials Change
# 20181004 PL Created

In [2]:
# example on path to avro-file
# /rawdata/trackcircuits/lysaker/bn-maintenance40-eh/trackcircuitlysaker/3/2018/08/29/08/02/51.avro
# /rawdata/trackcircuits/<station>/bn-maintenance40-eh/trackciruuit<station>/<partition>/<year>/<month>/<day>/<hour>/<minute>/<AVRO-FILE>

# Batch name of trained models/norm matrix
folder =  dbutils.widgets.get("folder")
#folder = 'sep2018_nov2018'

#import datetime
#year = datetime.datetime.today().year
#month = datetime.datetime.today().month
#day = datetime.datetime.today().day

# Time parameters
day =  dbutils.widgets.get("day")
month =  dbutils.widgets.get("month")
year =  dbutils.widgets.get("year")
#day = '28'

# Stations that should be included
stations = ["lysaker", "skoyen", "nationaltheatret", "sandvika", "asker"]

# doing left padding to fit format of datalake paths
day = day.zfill(2)
month = month.zfill(2)

path_to_daily_data = "{}/{}/{}".format(year,month,day)

# creating list of paths
paths = []
for station in stations: 
  path = "mnt/root/rawdata/trackcircuits/{}/bn-maintenance40-eh/trackcircuit{}/*/{}/*/*/*".format(station, station, path_to_daily_data)
  paths.append(path)

  


In [3]:
import sys
import json
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import from_json, col, input_file_name
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType

# Reading data from datalake
df = spark.read.format("com.databricks.spark.avro").load(paths)

# Defining the schema the data is on 
## In the data lake there are more columns and some columns that are named exactly the same as the ones we want to extract but in all caps or in all none-caps. Spark is case-insensitive but by defining the JSON-schema we can get only the data we want
json_schema = StructType([
    StructField("currentDirectionKind", StringType(), True),
    StructField("eventAt", DoubleType(), True),
    StructField("measurement", LongType(), True),
    StructField("tcid", StringType(), True)])

#Exctracting body from binary                                                       
df = df.withColumn('body', df['body'].cast('string'))
body = df.select('body')
data = body.withColumn('body', from_json(col('body'), json_schema)).select('body.*')


# Adding station column (this is done by choosing the fifth element in the filepath) and this approach works for all TC not in "SmallContributors"-paths

data = data.withColumn("filename", input_file_name())

split_col = pyspark.sql.functions.split(data['filename'], '/')
data = data.withColumn('Station', split_col.getItem(5))

data = data.drop("filename")
error_code = data.filter("measurement > 32000") # A value of 32764 implies that the sensor sends out some error code and we are therefore filter these. We will have to look into it at a later period
data = data.filter("measurement < 32000") # A value of 32764 implies that the sensor sends out some error code and we are therefore filter these. We will have to look into it at a later period




In [4]:
# Applying deadband

#print("Number of rows original is {}".format(data.count()))

#Fixing the column names (adding columns with the same name as the old data)
data = data.withColumn("orgTimestamp", data.eventAt)
data = data.withColumn("Measurement", data.measurement)
data = data.withColumn("syncTimestamp", (F.round(data.eventAt/250)*250).cast("double"))
data = data.withColumn("TrackCircuitId", data.tcid)
data = data.withColumn("Current", data.currentDirectionKind)

# Applying a deadbandfilter. All changes in values are permitted to pass
windowSpec = \
  Window \
    .partitionBy("Station","TrackCircuitId", "Current") \
    .orderBy("orgTimestamp")

data = data.withColumn("prevval", F.lag(data.Measurement).over(windowSpec))
data = data.filter("Measurement != prevval or prevval is null")
data = data.drop("prevval")

data = data.select("Station","TrackCircuitId","Measurement","orgTimestamp", "syncTimestamp","Current").cache()


In [5]:
# Syncing time and removing duplicates

# This is done by the following steps
# 1. Timestamp is rounded to nearest 250 ms. If the value is alone on that timestamp it is chosen
# 2. Remove duplicate values, i.e. same measurement on same rounded timestamp for same trackcircuit
# 3. If multiple values on same rounded timestamp stil exist. The absolute differance between rounded timestamp 
#    and the original timestamp is calculated. The value with the smallest difference is selected.
# 4. If multiple values with same differance exist. the smallest of these are selected

#1 Done in previous snippets
# data = data.withColumn("syncTimestamp", (F.round(data.orgTimestamp/250)*250).cast.("double"))
#print("Got {} rows before syncing".format(data.count()))

#2
data = data.dropDuplicates(["syncTimestamp","Station","TrackCircuitId","Current","Measurement"]) # .dropDuplicates have implicit keep = first

# Print that was used during development
#print("Got {} rows after removing synced duplicates".format(data.count()))

# 3 # 4
data = data.withColumn("diffTimestamps",  F.abs(data["orgTimestamp"]-data["syncTimestamp"]))
data = data.orderBy(["diffTimestamps", "Measurement"], ascending =[True,True]) # Arranging the dataframe in the order we want
data = data.dropDuplicates(["syncTimestamp","Station","TrackCircuitId","Current"]) # .dropDuplicates have implicit keep = first

#print("Got {} rows after removing synced duplicate with different measurements".format(data.count()))
data = data.drop("diffTimestamps")
data = data.withColumnRenamed('syncTimestamp','Timestamp')


In [6]:
# Pivoting RC and FC to one row and forwardfill the values

# print("Dataframe contains {} number of rows before pivoting".format(df.count()))

# Creating the pivoted dataframe
df_pivot = data.groupBy("Station","TrackCircuitId", "Timestamp").pivot("Current", ["FC","RC"]).sum("Measurement")

# Renaming columns
df_pivot = df_pivot.withColumnRenamed("RC", "Measurement_RC")
df_pivot = df_pivot.withColumnRenamed("FC", "Measurement_FC")
#print("Dataframe contains {} number of rows after pivoting".format(df_pivot.count()))


# Forwardfilling NULL-values
### Using window function for easy implementation

# define the window
windowSpec = \
  Window \
    .partitionBy("Station","TrackCircuitId") \
    .orderBy("Timestamp") \
    .rowsBetween(-sys.maxsize, 0) # sys.maxsize is utilize to start from beginning from each partition

# define the forward-filled column
filled_column_RC = F.last(df_pivot['Measurement_RC'], ignorenulls=True).over(windowSpec)
filled_column_FC = F.last(df_pivot['Measurement_FC'], ignorenulls=True).over(windowSpec)

# do the fill 
df_pivot = df_pivot.withColumn('Measurement_RC',  filled_column_RC)
df_pivot = df_pivot.withColumn('Measurement_FC',  filled_column_FC)


In [7]:
frequence = 4 # Data is synced to 4 hz, therefore we can say that that 1 seconds contains 4 measurements

# Adding some additional information based on next row

windowSpec = \
  Window \
    .partitionBy("Station","TrackCircuitId") \
    .orderBy("Timestamp") \

endTimestamp = F.lead(df_pivot['Timestamp'],1).over(windowSpec)

df_pivot = df_pivot.withColumn("End", endTimestamp)
df_pivot = df_pivot.withColumn("Deltatime", df_pivot["End"]-df_pivot["Timestamp"])
df_pivot = df_pivot.withColumn("DeltatimeSeconds", df_pivot["Deltatime"]/1000) #Deltatime is given in milliseconds
df_pivot = df_pivot.withColumn("DeltaWeights", df_pivot["DeltatimeSeconds"]*frequence) #Weights for use in later calculation of average and standard deviations


In [8]:
# New version, 20180918 (built ontop Mattis StreamAnalytics code) to detect wheter or not a track circuit is occupied or free
## This could be replaced by information that is collected from Stream Analytics-calculations, however where to get these calculations are not clear today

# Code used in development to start fresh
#df_pivot = df_pivot.select("Station","TrackCircuitId","Timestamp","Measurement_FC","Measurement_RC","End","Deltatime","DeltatimeSeconds")

# Specifying states
TC_OCCUPIED_STATE = "Occupied"
TC_FREE_STATE = "Free"
TC_UNKNOWN_STATE = "Unknown"
TC_ARRIVING_STATE = 'Arriving'
TC_DEPARTING_STATE = 'Departing'
TC_UNCERTAIN_STATE = 'Uncertain'

# Setting state of RC
border_high_rc = 140
border_low_rc = 120
df_pivot = df_pivot.withColumn("State_RC", 
                               F.when(df_pivot["Measurement_RC"]< border_low_rc, TC_OCCUPIED_STATE)
                               .when(df_pivot["Measurement_RC"] > border_high_rc, TC_FREE_STATE)
                               .otherwise(TC_UNCERTAIN_STATE)
                              )

# Setting state of FC 
temppath = '/mnt/root/ml/trackcircuits/data/threshold_fc/{}/'.format(folder)
threshold_fc = spark.read.format("com.databricks.spark.avro").option("basePath", temppath).load(temppath)

df_pivot= df_pivot.join(threshold_fc.select("Station","TrackCircuitId", "border_low_FC", "border_high_FC"), ["Station","TrackCircuitId"])
df_pivot = df_pivot.withColumn("State_FC",
                               F.when(df_pivot["Measurement_FC"] < df_pivot["border_low_FC"], TC_FREE_STATE)
                               .when(df_pivot["Measurement_FC"] > df_pivot["border_high_FC"], TC_OCCUPIED_STATE)
                               .otherwise(TC_UNCERTAIN_STATE))

# Filling out the first round of states (Free, Occupied, Unknown) based on RC and FC state
df_pivot = df_pivot.withColumn("State1",
                               F.when(df_pivot["State_RC"] == df_pivot["State_FC"], df_pivot["State_RC"])
                               .otherwise(TC_UNKNOWN_STATE)
                              )


In [9]:
# Code to splitt up Unknown-state into Departing-, Arriving- and Unknown-states based on the track circuit behaviour before and after an unknown period

df_pivot = df_pivot.select("Station","TrackCircuitId","Timestamp","Measurement_FC","Measurement_RC","End","Deltatime","DeltatimeSeconds","DeltaWeights","State_RC","State_FC","State1")

# Checking the line of combination of states to set states Arriving and Departing
# In order to do that we need to attach information from last state and next state for each unknown state (and an unknown state can span for a (always unknown) number of points)

# Adding some additional information based on next row

windowSpec = \
  Window \
    .partitionBy("Station","TrackCircuitId") \
    .orderBy("Timestamp") \

prev_state = F.lag(df_pivot['State1'],1).over(windowSpec)
next_state = F.lead(df_pivot["State1"],1).over(windowSpec)

df_pivot = df_pivot.withColumn("previousState", prev_state)
df_pivot = df_pivot.withColumn("nextState", next_state)

df_pivot = df_pivot.withColumn("previousPlateauState",
                               F.when(F.isnull(df_pivot["previousState"]), df_pivot["State1"])
                               .when(df_pivot["State1"]==df_pivot["previousState"], None)
                               .otherwise(df_pivot["previousState"]))
df_pivot = df_pivot.withColumn("nextPlateauState",
                               F.when(F.isnull(df_pivot["previousState"]), df_pivot["State1"])
                               .when(df_pivot["State1"]==df_pivot["nextState"], None)
                               .otherwise(df_pivot["nextState"]))

# Forwardfilling NULL-values
### Using window function for easy implementation

# define the window for previous state
windowSpec = \
  Window \
    .partitionBy("Station","TrackCircuitId") \
    .orderBy("Timestamp") \
    .rowsBetween(-sys.maxsize, 0) # sys.maxsize is utilize to start from beginning from each partition

# define the forward-filled column
previousPlataeuState = F.last(df_pivot['previousPlateauState'], ignorenulls=True).over(windowSpec)

# do the fill 
df_pivot = df_pivot.withColumn('previousPlateauState',  previousPlataeuState)


# define the window for next state
  # Order the data in reverse order from the previous state
windowSpec = \
  Window \
    .partitionBy("Station","TrackCircuitId") \
    .orderBy(F.col("Timestamp").desc()) \
    .rowsBetween(-sys.maxsize, 0) # sys.maxsize is utilize to start from beginning from each partition

# define the back-filled column
nextPlateauState = F.last(df_pivot['nextPlateauState'], ignorenulls=True).over(windowSpec)

# do the fill 
df_pivot = df_pivot.withColumn('nextPlateauState',  nextPlateauState)



# Setting final state based on the sequence of states. 
# Final states is one of the following Free, Occupied, Unknown, Arriving, Departing

df_pivot = df_pivot.withColumn("State", 
                               F.when((df_pivot["State1"]== TC_FREE_STATE) | (df_pivot["State1"] == TC_OCCUPIED_STATE), df_pivot["State1"]) # df["State"] kan only be Free, Occupied or Unknown
                               .when((df_pivot["previousPlateauState"] == TC_FREE_STATE) & (df_pivot["nextPlateauState"]== TC_OCCUPIED_STATE), TC_ARRIVING_STATE) # When an unknown is preceeded by an Free and succeded by an Occupied it is an arriving state
                               .when((df_pivot["previousPlateauState"] == TC_OCCUPIED_STATE) & (df_pivot["nextPlateauState"]== TC_FREE_STATE), TC_DEPARTING_STATE)# When an unknown is preceeded by an Occupied and succeded by an Free it is an Departing state
                               .otherwise(TC_UNKNOWN_STATE) 
                              )


# Triming the dataset, removing the first and the last line of each track circuit to avoid null-values due to lead/lag-functions
df_pivot = df_pivot.filter("nextState is not null and previousState is not null")

df_pivot = df_pivot.select("Station","TrackCircuitId","Timestamp","Measurement_FC","Measurement_RC","End","Deltatime","DeltatimeSeconds","DeltaWeights","State_RC","State_FC","State")


In [10]:
# Importing the norm matrix that is to be used
# The norm matrix is related to the data used in training of a model (i.e. a batch name) 
# There will therefore exist multiple normalization matricies (and models)


norm_matrix_path = "/mnt/root/ml/trackcircuits/data/norm_matrix/{}/".format(folder)
norm_matrix = spark.read.format("com.databricks.spark.avro").option("basePath", norm_matrix_path).load(norm_matrix_path)

df_pivot = df_pivot.join(norm_matrix, on =["Station","TrackCircuitId","State"], how = "left")

# Calculate normalized RC and FC values as (value - mean) / stddev
df_pivot = df_pivot.withColumn("Measurement_FC_norm", (df_pivot["Measurement_FC"] - df_pivot["wAvgFC"]) / df_pivot["wStdFC"])
df_pivot = df_pivot.withColumn("Measurement_RC_norm", (df_pivot["Measurement_RC"] - df_pivot["wAvgRC"]) / df_pivot["wStdRC"])


In [11]:
# Creating the plateaus. This is done by generating a plateauID based on matching the previous stat to current state for each track circuit

# Defining a window 
windowSpec = \
  Window \
    .partitionBy("Station","TrackCircuitId") \
    .orderBy("Timestamp") \

# Creating a previous state
state = F.lag(df_pivot['State'],1).over(windowSpec)
df_pivot = df_pivot.withColumn("previousState", state)

# Generating a ID if previousstate != state for each row
df_pivot = df_pivot.withColumn("plateauId", 
                               F.when(F.isnan(df_pivot["previousState"]) ,F.monotonically_increasing_id()) # First row gets first ID
                               .when(df_pivot["State"]==df_pivot["PreviousState"], None) # No id generated for rows following start of plateau
                               .otherwise(F.monotonically_increasing_id())) # ID generated for first measurement of new plateau


# Forwardfilling NULL-values
### Using window function for easy implementation

# define the window
windowSpec = \
  Window \
    .partitionBy("Station","TrackCircuitId") \
    .orderBy("Timestamp") \
    .rowsBetween(-sys.maxsize, 0) # sys.maxsize is utilize to start from beginning from each partition

# define the forward-filled column and filling
plateauId = F.last(df_pivot['plateauId'], ignorenulls=True).over(windowSpec)
df_pivot = df_pivot.withColumn('plateauId',  plateauId)


In [12]:
# Defining some functioned used to minimize code

def wavg (values, weights):
  wavg = F.sum(values*weights)/F.sum(weights)
  return wavg

def wstd (values, wavg, weights): # Here we need to pass wavg as well since it dont work with a call towards function wavg
  wv = F.sum(((values-wavg)**2)*weights) / F.sum(weights)
  wstd_biased = F.sqrt(wv)  
  wstd = F.when(F.sum(weights) == 1, wstd_biased).otherwise(wstd_biased * F.sum(weights) / (F.sum(weights) - 1))
  return wstd

In [13]:
# Calculation of values that descirbes plateaus

# 1. Min/Max/Avg for RC/FC
# 2. Std for RC/FC

# Code to start on clean snippet during development
df_pivot = df_pivot.select("TrackCircuitId","State","Timestamp","Measurement_FC","Measurement_RC","End","Deltatime","DeltatimeSeconds","DeltaWeights","State_RC","State_FC","wAvgFC","wAvgRC","wStdFC","wStdRC","count","Measurement_FC_norm","Measurement_RC_norm","Station","previousState","plateauId")


# 1.
# Creating group
tmpGrp = df_pivot.groupby("Station","TrackCircuitId","State","plateauId")

tmpBasic = tmpGrp.agg(F.min("Timestamp").alias("Start"), # Min timestamp is start of plateau
                      F.max("End").alias("End"), # Max end (timestamp) is end of plateau
                      (F.max("End") - F.min("Timestamp")).alias("Length"), # Difference between start and stop i length of plateau. protip: This should equal sum(Deltatime) F.sum("Deltatime")
                      F.count(F.lit(1)).alias("Count") # Number of observations in the plateau
                     ) 

# Calculation of min, max and weighted average
tmpRC = tmpGrp.agg(F.min("Measurement_RC_norm").alias("Min_RC"), # Minimum RC value
                   F.max("Measurement_RC_norm").alias("Max_RC"), # Maximum RC value
                   wavg(df_pivot["Measurement_RC_norm"], df_pivot["DeltaWeights"]).alias("Avg_RC")
                  )

tmpFC = tmpGrp.agg(F.min("Measurement_FC_norm").alias("Min_FC"), # Minimum FC value
                   F.max("Measurement_FC_norm").alias("Max_FC"), # Maximum FC value
                   wavg(df_pivot["Measurement_FC_norm"], df_pivot["DeltaWeights"]).alias("Avg_FC")
                  )


# 2.
# Joining average calculation on df in order to use it in calculation for weighted standard deviation
df_pivot = df_pivot.join(tmpRC.select("Station","TrackCircuitId","State","plateauId","Avg_RC"),
                         on = (["Station","TrackCircuitId","State","plateauId"]),
                         how = "left")
df_pivot = df_pivot.join(tmpFC.select("Station","TrackCircuitId","State","plateauId","Avg_FC"),
                         on = (["Station","TrackCircuitId","State","plateauId"]),
                         how = "left")


# Redoing the basis for norm_matrix to include weighted average 
tmpGrp = df_pivot.groupBy("Station","TrackCircuitId","State","plateauId")

# Calculation of weighted standard deviation
tmpStd = tmpGrp.agg(wstd(df_pivot["Measurement_RC_norm"], df_pivot["Avg_RC"], df_pivot["DeltaWeights"]).alias("Std_RC"),
                    wstd(df_pivot["Measurement_FC_norm"], df_pivot["Avg_FC"], df_pivot["DeltaWeights"]).alias("Std_FC")
                   )


## How are we supposed to calculate tilt in an efficient way????
# Tilt is left out as of now but will probably return after some work

df_plateau = tmpBasic.join(tmpRC, on = (["Station","TrackCircuitId","State","plateauId"]), how = "left") \
                     .join(tmpFC, on = (["Station","TrackCircuitId","State","plateauId"]), how = "left") \
                     .join(tmpStd, on = (["Station","TrackCircuitId","State","plateauId"]), how = "left") 



In [14]:
# Creating some features on daily level based on measurements
# E.g. min, max, std, and so on.

# Creation of daily dataset (one row per TC and day that exist in the data)

# 1. Unknown Ratio (both count and length)
# 2. Free average measurements RC/FC
# 3. Occupied average measurements RC/FC
# 4. Free standard deviation RC/FC
# 5. Occupied standart deviation RC/FC

# Code to start snippet clean during development
df_pivot = df_pivot.select("TrackCircuitId","State","Timestamp","Measurement_FC","Measurement_RC","End","Deltatime","DeltatimeSeconds","DeltaWeights","State_RC","State_FC","wAvgFC","wAvgRC","wStdFC","wStdRC","count","Measurement_FC_norm","Measurement_RC_norm","Station","previousState","plateauId")

# Defining a key for use in multiple places in this snippet
key = ["Station", "TrackCircuitId"]

# 1.

df_day_meas = df_pivot.groupBy(key).pivot("State",[TC_FREE_STATE, TC_OCCUPIED_STATE,TC_UNKNOWN_STATE,TC_ARRIVING_STATE,TC_DEPARTING_STATE])\
                                   .agg(F.count(F.lit(1)).alias("count"),
                                        F.sum("Deltatime").alias("length")
                                        )
 
df_day_meas = df_day_meas.withColumn("FREE_COUNT", df_day_meas["Free_count"])
df_day_meas = df_day_meas.withColumn("OCCUPIED_COUNT", df_day_meas["Occupied_count"])
df_day_meas = df_day_meas.withColumn("UNKNOWN_COUNT", df_day_meas["Unknown_count"])
df_day_meas = df_day_meas.withColumn("ARRIVING_COUNT", df_day_meas["Arriving_count"])
df_day_meas = df_day_meas.withColumn("DEPARTING_COUNT", df_day_meas["Departing_count"])
df_day_meas = df_day_meas.withColumn("UNKNOWN_LENGTH", df_day_meas["Unknown_length"])
df_day_meas = df_day_meas.withColumn("ARRIVING_LENGTH", df_day_meas["Arriving_length"])
df_day_meas = df_day_meas.withColumn("DEPARTING_LENGTH", df_day_meas["Departing_length"])
df_day_meas = df_day_meas.withColumn("TOTAL_COUNT", (F.coalesce(df_day_meas["FREE_COUNT"],F.lit(0)) +
                                                     F.coalesce(df_day_meas["OCCUPIED_COUNT"],F.lit(0)) + 
                                                     F.coalesce(df_day_meas["UNKNOWN_COUNT"],F.lit(0)) + 
                                                     F.coalesce(df_day_meas["ARRIVING_COUNT"],F.lit(0)) +
                                                     F.coalesce(df_day_meas["DEPARTING_COUNT"],F.lit(0))))

df_day_meas = df_day_meas.withColumn("UNKNOWN_RATIO", df_day_meas["UNKNOWN_COUNT"]/df_day_meas["TOTAL_COUNT"])
df_day_meas = df_day_meas.withColumn("ARRIVING_RATIO", (df_day_meas["ARRIVING_COUNT"] / df_day_meas["TOTAL_COUNT"]))
df_day_meas = df_day_meas.withColumn("DEPARTING_RATIO", (df_day_meas["DEPARTING_COUNT"] / df_day_meas["TOTAL_COUNT"]))

columns_included = ["Station","TrackCircuitId","FREE_COUNT","OCCUPIED_COUNT","UNKNOWN_COUNT","ARRIVING_COUNT", "DEPARTING_COUNT", "TOTAL_COUNT","UNKNOWN_RATIO","ARRIVING_RATIO","DEPARTING_RATIO","UNKNOWN_LENGTH","ARRIVING_LENGTH","DEPARTING_LENGTH"]

df_day_meas = df_day_meas.select(columns_included).fillna(0, subset = ["FREE_COUNT", "OCCUPIED_COUNT", "UNKNOWN_COUNT", "ARRIVING_COUNT", 
                                                                       "DEPARTING_COUNT", "TOTAL_COUNT", "UNKNOWN_RATIO","ARRIVING_RATIO","DEPARTING_RATIO",
                                                                       "UNKNOWN_LENGTH","ARRIVING_LENGTH","DEPARTING_LENGTH"])


# 2.
tmpGrp = df_pivot.filter("state = 'Free'").groupby(key)
tmpFree = tmpGrp.agg(wavg(df_pivot["Measurement_RC_norm"], df_pivot["Deltaweights"]).alias("RC_F_AVG_MEASUREMENT"),
                     wavg(df_pivot["Measurement_FC_norm"], df_pivot["Deltaweights"]).alias("FC_F_AVG_MEASUREMENT")
                    )

# 3.
tmpGrp = df_pivot.filter("state = 'Occupied'").groupby(key)
tmpOccupied = tmpGrp.agg(wavg(df_pivot["Measurement_RC_norm"], df_pivot["Deltaweights"]).alias("RC_O_AVG_MEASUREMENT"),
                         wavg(df_pivot["Measurement_FC_norm"], df_pivot["Deltaweights"]).alias("FC_O_AVG_MEASUREMENT")
                        )

df_day_meas = df_day_meas.join(tmpFree, on = key, how = "left")
df_day_meas = df_day_meas.join(tmpOccupied, on = key, how = "left")


# Joining average calculation on df in order to use it in calculation for weighted standard deviation
df_pivot = df_pivot.join(tmpFree.select("Station","TrackCircuitId","RC_F_AVG_MEASUREMENT", "FC_F_AVG_MEASUREMENT"),
                         on = key,
                         how = "left")
df_pivot = df_pivot.join(tmpOccupied.select("Station","TrackCircuitId","RC_O_AVG_MEASUREMENT", "FC_O_AVG_MEASUREMENT"),
                         on = key,
                         how = "left")


# Calculation of weighted standard deviation
# 4. 
tmpGrp = df_pivot.filter("state = 'Free'").groupby(key)
tmpFree = tmpGrp.agg(wstd(df_pivot["Measurement_RC_norm"], df_pivot["RC_F_AVG_MEASUREMENT"], df_pivot["DeltaWeights"]).alias("RC_F_STD_MEASUREMENT"),
                     wstd(df_pivot["Measurement_FC_norm"], df_pivot["FC_F_AVG_MEASUREMENT"], df_pivot["DeltaWeights"]).alias("FC_F_STD_MEASUREMENT")
                    )
# 5.
tmpGrp = df_pivot.filter("state = 'Occupied'").groupby(key)
tmpOccupied = tmpGrp.agg(wstd(df_pivot["Measurement_RC_norm"], df_pivot["RC_O_AVG_MEASUREMENT"], df_pivot["DeltaWeights"]).alias("RC_O_STD_MEASUREMENT"),
                         wstd(df_pivot["Measurement_FC_norm"], df_pivot["FC_O_AVG_MEASUREMENT"], df_pivot["DeltaWeights"]).alias("FC_O_STD_MEASUREMENT")
                        )

df_day_meas = df_day_meas.join(tmpFree, on = key, how = "left")
df_day_meas = df_day_meas.join(tmpOccupied, on = key, how = "left")




In [15]:
# Calculating features related to plateaus

TC_OCCUPIED_STATE = "Occupied"
TC_FREE_STATE = "Free"

key = ["Station","TrackCircuitId"]

df_day_plat = df_plateau\
              .withColumn("diff_within_plateau_rc",df_plateau["Max_RC"] -df_plateau["Min_RC"])\
              .withColumn("diff_within_plateau_fc",df_plateau["Max_FC"] -df_plateau["Min_FC"])\
              .groupBy(key)\
              .pivot("State", ["Free", "Occupied","Unknown","Arriving", "Departing"])\
              .agg(F.min("Std_RC").alias("min_std_rc"),
                   F.min("Std_FC").alias("min_std_fc"),
                   F.max("Std_RC").alias("max_std_rc"),
                   F.max("Std_FC").alias("max_std_fc"),
                   F.stddev("Std_RC").alias("std_std_rc"),
                   F.stddev("Std_FC").alias("std_std_fc"),
                   F.max("Avg_RC").alias("max_avg_rc"),
                   F.min("Avg_RC").alias("min_avg_rc"),
                   F.max("Avg_FC").alias("max_avg_fc"),
                   F.min("Avg_FC").alias("min_avg_fc"),
                   F.max("diff_within_plateau_rc").alias("max_diff_max_min_rc"),
                   F.min("diff_within_plateau_rc").alias("min_diff_max_min_rc"),
                   F.max("diff_within_plateau_fc").alias("max_diff_max_min_fc"),
                   F.min("diff_within_plateau_fc").alias("min_diff_max_min_fc"),
                   F.avg("Length").alias("avg_length"),
                   F.max("Length").alias("max_length")
                  )


  
df_day_plat = df_day_plat.withColumn("Free_diff_maxmin_avg_rc",
                                     df_day_plat["Free_max_avg_rc"]-df_day_plat["Free_min_avg_rc"])
df_day_plat = df_day_plat.withColumn("Free_diff_maxmin_avg_fc",
                                     df_day_plat["Free_max_avg_fc"]-df_day_plat["Free_min_avg_fc"])
df_day_plat = df_day_plat.withColumn("Occupied_diff_maxmin_avg_rc",
                                     df_day_plat["Occupied_max_avg_rc"]-df_day_plat["Occupied_min_avg_rc"])
df_day_plat = df_day_plat.withColumn("Occupied_diff_maxmin_avg_fc",
                                     df_day_plat["Occupied_max_avg_fc"]-df_day_plat["Occupied_min_avg_fc"])


df_day_plat = df_day_plat.select("Station","TrackCircuitId",
                                 # Features related to STD within a plateau
                                 "Free_min_std_rc", "Free_max_std_rc", "Free_std_std_rc",
                                 "Free_min_std_fc", "Free_max_std_fc", "Free_std_std_fc",
                                 "Occupied_min_std_rc", "Occupied_max_std_rc", "Occupied_std_std_rc",
                                 "Occupied_min_std_fc", "Occupied_max_std_fc", "Occupied_std_std_fc",
                                 # Features related to differences between plateau in avg
                                 "Free_diff_maxmin_avg_rc","Free_diff_maxmin_avg_fc",
                                 "Occupied_diff_maxmin_avg_rc","Occupied_diff_maxmin_avg_fc",
                                 # Feature related to differences within a plateau
                                 "Free_max_diff_max_min_rc", "Free_max_diff_max_min_fc",
                                 "Free_min_diff_max_min_rc", "Free_min_diff_max_min_fc",
                                 "Occupied_max_diff_max_min_rc", "Occupied_max_diff_max_min_fc",
                                 "Occupied_min_diff_max_min_rc", "Occupied_min_diff_max_min_fc",
                                 # Feature related to lenght of plateau
                                 "Arriving_avg_length", "Departing_avg_length", "Unknown_avg_length",
                                 "Free_avg_length","Occupied_avg_length",
                                 "Arriving_max_length", "Departing_max_length", "Unknown_max_length",
                                 "Free_max_length","Occupied_max_length"
                                ).fillna(0, subset = ["Arriving_avg_length","Departing_avg_length","Unknown_avg_length","Free_avg_length",
                                                      "Occupied_avg_length",
                                                      "Arriving_max_length","Departing_max_length","Unknown_max_length","Free_max_length",
                                                      "Occupied_max_length"])

df_day_plat = df_day_plat.withColumnRenamed("Free_min_std_rc", "RC_F_MIN_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Free_max_std_rc", "RC_F_MAX_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Free_std_std_rc", "RC_F_STD_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Free_min_std_fc", "FC_F_MIN_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Free_max_std_fc", "FC_F_MAX_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Free_std_std_fc", "FC_F_STD_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_min_std_rc", "RC_O_MIN_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_max_std_rc", "RC_O_MAX_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_std_std_rc", "RC_O_STD_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_min_std_fc", "FC_O_MIN_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_max_std_fc", "FC_O_MAX_STD_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_std_std_fc", "FC_O_STD_STD_PLATEAU")

df_day_plat = df_day_plat.withColumnRenamed("Free_diff_maxmin_avg_rc", "RC_F_DIFF_MAXAVG_MINAVG_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Free_diff_maxmin_avg_fc", "FC_F_DIFF_MAXAVG_MINAVG_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_diff_maxmin_avg_rc", "RC_O_DIFF_MAXAVG_MINAVG_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_diff_maxmin_avg_fc", "FC_O_DIFF_MAXAVG_MINAVG_PLATEAU")

df_day_plat = df_day_plat.withColumnRenamed("Free_max_diff_max_min_rc", "RC_F_MAXDIFF_MAX_MIN_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Free_max_diff_max_min_fc", "FC_F_MAXDIFF_MAX_MIN_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Free_min_diff_max_min_rc", "RC_F_MINDIFF_MAX_MIN_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Free_min_diff_max_min_fc", "FC_F_MIMDIFF_MAX_MIN_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_max_diff_max_min_rc", "RC_O_MAXDIFF_MAX_MIN_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_max_diff_max_min_fc", "FC_O_MAXDIFF_MAX_MIN_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_min_diff_max_min_rc", "RC_O_MINDIFF_MAX_MIN_PLATEAU")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_min_diff_max_min_fc", "FC_O_MIMDIFF_MAX_MIN_PLATEAU")

df_day_plat = df_day_plat.withColumnRenamed("Arriving_avg_length", "ARRIVING_AVG_LENGTH")
df_day_plat = df_day_plat.withColumnRenamed("Departing_avg_length", "DEPARTING_AVG_LENGTH")
df_day_plat = df_day_plat.withColumnRenamed("Unknown_avg_length", "UNKNOWN_AVG_LENGTH")
df_day_plat = df_day_plat.withColumnRenamed("Free_avg_length", "FREE_AVG_LENGTH")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_avg_length", "OCCUPIED_AVG_LENGTH")

df_day_plat = df_day_plat.withColumnRenamed("Arriving_max_length", "ARRIVING_MAX_LENGTH")
df_day_plat = df_day_plat.withColumnRenamed("Departing_max_length", "DEPARTING_MAX_LENGTH")
df_day_plat = df_day_plat.withColumnRenamed("Unknown_max_length", "UNKNOWN_MAX_LENGTH")
df_day_plat = df_day_plat.withColumnRenamed("Free_max_length", "FREE_MAX_LENGTH")
df_day_plat = df_day_plat.withColumnRenamed("Occupied_max_length", "OCCUPIED_MAX_LENGTH")


In [16]:
# Calculation of passages

df_passage = df_plateau.filter("State in ('Free','Occupied')")

windowSpec = \
  Window \
    .partitionBy("Station","TrackCircuitId") \
    .orderBy("Start") \

prevstate = F.lag(df_passage['State'],1).over(windowSpec)
df_passage = df_passage.withColumn("previousState", prevstate)

nextstate = F.lead(df_passage['State'],1).over(windowSpec)
df_passage = df_passage.withColumn("nextState", nextstate)

prevAvgRC = F.lag(df_passage['Avg_RC'],1).over(windowSpec)
prevAvgFC = F.lag(df_passage['Avg_FC'],1).over(windowSpec)
df_passage = df_passage.withColumn("previousAvg_RC", prevAvgRC)
df_passage = df_passage.withColumn("previousAvg_FC", prevAvgFC)

nextAvgRC = F.lead(df_passage['Avg_RC'],1).over(windowSpec)
nextAvgFC = F.lead(df_passage['Avg_FC'],1).over(windowSpec)
df_passage = df_passage.withColumn("nextAvg_RC", nextAvgRC)
df_passage = df_passage.withColumn("nextAvg_FC", nextAvgFC)

df_passage = df_passage.filter("State = 'Occupied' and nextState = 'Free' and previousState ='Free'")

df_passage = df_passage.withColumn("RC_PASS_AVGDIFF_BEF_AFT", F.abs(df_passage["previousAvg_RC"] -  df_passage["nextAvg_RC"]))
df_passage = df_passage.withColumn("FC_PASS_AVGDIFF_BEF_AFT", F.abs(df_passage["previousAvg_FC"] -  df_passage["nextAvg_FC"]))
df_passage = df_passage.withColumn("RC_PASS_AVGDIFF_BEF_PASS", df_passage["previousAvg_RC"] -  df_passage["Avg_RC"])
df_passage = df_passage.withColumn("FC_PASS_AVGDIFF_BEF_PASS", df_passage["previousAvg_FC"] -  df_passage["Avg_FC"])
df_passage = df_passage.withColumn("RC_PASS_AVGDIFF_PASS_AFT", df_passage["Avg_RC"] -  df_passage["nextAvg_RC"])
df_passage = df_passage.withColumn("FC_PASS_AVGDIFF_PASS_AFT", df_passage["Avg_FC"] -  df_passage["nextAvg_FC"])

df_passage = df_passage.select("Station","TrackCircuitId",
                               "RC_PASS_AVGDIFF_BEF_AFT","FC_PASS_AVGDIFF_BEF_AFT",
                               "RC_PASS_AVGDIFF_BEF_PASS","FC_PASS_AVGDIFF_BEF_PASS",
                               "RC_PASS_AVGDIFF_PASS_AFT","FC_PASS_AVGDIFF_PASS_AFT"
                              )


df_day_pass = df_passage\
              .groupBy("Station","TrackCircuitId")\
              .agg(# Maxium calulations
                   F.max("RC_PASS_AVGDIFF_BEF_AFT").alias("RC_PASS_MAXDIFF_AVG_BEF_AFT"),
                   F.max("FC_PASS_AVGDIFF_BEF_AFT").alias("FC_PASS_MAXDIFF_AVG_BEF_AFT"),
                   F.max("RC_PASS_AVGDIFF_BEF_PASS").alias("RC_PASS_MAXDIFF_AVG_BEF_PASS"),
                   F.max("FC_PASS_AVGDIFF_BEF_PASS").alias("FC_PASS_MAXDIFF_AVG_BEF_PASS"),
                   F.max("RC_PASS_AVGDIFF_PASS_AFT").alias("RC_PASS_MAXDIFF_AVG_PASS_AFT"),
                   F.max("FC_PASS_AVGDIFF_PASS_AFT").alias("FC_PASS_MAXDIFF_AVG_PASS_AFT"),
                   # Minimum calc
                   F.min("RC_PASS_AVGDIFF_BEF_AFT").alias("RC_PASS_MINDIFF_AVG_BEF_AFT"),
                   F.min("FC_PASS_AVGDIFF_BEF_AFT").alias("FC_PASS_MINDIFF_AVG_BEF_AFT"),
                   F.min("RC_PASS_AVGDIFF_BEF_PASS").alias("RC_PASS_MINDIFF_AVG_BEF_PASS"),
                   F.min("FC_PASS_AVGDIFF_BEF_PASS").alias("FC_PASS_MINDIFF_AVG_BEF_PASS"),
                   F.min("RC_PASS_AVGDIFF_PASS_AFT").alias("RC_PASS_MINDIFF_AVG_PASS_AFT"),
                   F.min("FC_PASS_AVGDIFF_PASS_AFT").alias("FC_PASS_MINDIFF_AVG_PASS_AFT"),
                   # Standard deviation calc
                   F.stddev("RC_PASS_AVGDIFF_BEF_AFT").alias("RC_PASS_STDDIFF_AVG_BEF_AFT"),
                   F.stddev("FC_PASS_AVGDIFF_BEF_AFT").alias("FC_PASS_STDDIFF_AVG_BEF_AFT"),
                   F.stddev("RC_PASS_AVGDIFF_BEF_PASS").alias("RC_PASS_STDDIFF_AVG_BEF_PASS"),
                   F.stddev("FC_PASS_AVGDIFF_BEF_PASS").alias("FC_PASS_STDDIFF_AVG_BEF_PASS"),
                   F.stddev("RC_PASS_AVGDIFF_PASS_AFT").alias("RC_PASS_STDDIFF_AVG_PASS_AFT"),
                   F.stddev("FC_PASS_AVGDIFF_PASS_AFT").alias("FC_PASS_STDDIFF_AVG_PASS_AFT")
                  )



In [17]:
# Writing data to files in order to maintain history and make it easier to back-etrack

# Writing preprocessed data to AVRO-files
cols = ['Station','TrackCircuitId','State','State_RC','State_FC','Timestamp','End','Deltatime','DeltatimeSeconds','DeltaWeights','Measurement_FC','Measurement_RC','Measurement_FC_norm','Measurement_RC_norm','wAvgFC','wAvgRC','wStdFC','wStdRC','count']

writepath = 'mnt/root/ml/trackcircuits/data/daily/{}/{}/{}/{}/preprocessed'.format(year,month,day,folder)
df_pivot.select(cols).write.partitionBy("Station").format("com.databricks.spark.avro").save(writepath)

# Writing plateau data to AVRO-files
writepath = 'mnt/root/ml/trackcircuits/data/daily/{}/{}/{}/{}/plateau'.format(year,month,day,folder)
df_plateau.write.partitionBy("Station").format("com.databricks.spark.avro").save(writepath)

# Writing passages data to AVRO-files
writepath = 'mnt/root/ml/trackcircuits/data/daily/{}/{}/{}/{}/passage'.format(year,month,day,folder)
df_passage.coalesce(1).write.partitionBy("Station").format("com.databricks.spark.avro").save(writepath)

# Writing feature from measurement data to AVRO-files
writepath = 'mnt/root/ml/trackcircuits/data/daily/{}/{}/{}/{}/feature_measurement'.format(year,month,day,folder)
df_day_meas.coalesce(1).write.partitionBy("Station").format("com.databricks.spark.avro").save(writepath)

# Writing feature from plateaus data to AVRO-files
writepath = 'mnt/root/ml/trackcircuits/data/daily/{}/{}/{}/{}/feature_plateau'.format(year,month,day,folder)
df_day_plat.coalesce(1).write.partitionBy("Station").format("com.databricks.spark.avro").save(writepath)

# Writing feature from passages data to AVRO-files
writepath = 'mnt/root/ml/trackcircuits/data/daily/{}/{}/{}/{}/feature_passage'.format(year,month,day,folder)
df_day_pass.coalesce(1).write.partitionBy("Station").format("com.databricks.spark.avro").save(writepath)

# Writing number of error codes in data set 
writepath = 'mnt/root/ml/trackcircuits/data/daily/{}/{}/{}/{}/error_codes'.format(year,month,day,folder)
error_code.groupBy("Station").count().coalesce(1).write.format("csv").option("Header", True).save(writepath)